In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv('../../../.env')


topic = 'Chess'

title_prompt = PromptTemplate.from_template(
    "Write a great title for a story about {topic}"
)

story_prompt = PromptTemplate.from_template(
    """You are a writer. Given the title of story, it is your job to write a story for that title.

Title: {title}"""
)

review_prompt = PromptTemplate.from_template(
    """You are a critic. Given a story, it is your job to write a review for that story.

Title: {title}
Story: {story}"""
)

llm = ChatOpenAI()

title_chain = title_prompt | llm | StrOutputParser()
story_chain = story_prompt | llm | StrOutputParser()
review_chain = review_prompt | llm | StrOutputParser()
chain = ({"title": title_chain}
         | RunnablePassthrough.assign(story=story_chain)
         | RunnablePassthrough.assign(review=review_chain))

if topic:
    result = chain.invoke({"topic": topic})

In [10]:
result

{'title': '"The Grand Game: Unveiling the Mind-Bending Strategies and Intense Battles of Chess"',
 'story': "Once upon a time in a small village nestled among rolling hills, there lived a young boy named Oliver. Oliver was fascinated by chess, a game that had been played for centuries and was beloved by many in the village. Every day, he would watch the elders gather in the town square, their eyes sparkling with excitement and their minds focused on the grand game.\n\nOliver longed to unravel the mysteries of chess, to understand the mind-bending strategies and intense battles that unfolded on the checkered board. He listened attentively to the tales of legendary chess matches, where players outsmarted one another with cunning moves and brilliant tactics. The more he heard, the more his curiosity grew, until he couldn't resist the urge to learn the game himself.\n\nWith a heart full of determination, Oliver approached the village chess master, Mr. Jenkins, renowned for his exceptional 

In [138]:
from langchain.chains import LLMChain, SequentialChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from operator import itemgetter
import requests
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Optional, Sequence
from langchain.chains.openai_functions import create_structured_output_runnable
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

load_dotenv('../../../.env')

BASE_URL = 'http://localhost:9000'

llm = ChatOpenAI()


class RelevantCollection(BaseModel):
    collection_id: str
    # id: str
    # description: str = Field(..., description="Descriptions of the collection")

RelevantCollections = Sequence[RelevantCollection]

class Property(BaseModel): 
    name: str
    type: str

class Collection(BaseModel):
    id: str
    description: str
    geometrytype: str
    properties: Sequence[Property]

class CollectionDescription(BaseModel):
    collection_id: str
    description: str = Field(..., description="Descriptions of the collection")
    geometrytype: str
    relevant_properties: Sequence[str]


CollectionDescriptions = Sequence[CollectionDescription]


def fetch_collections(collection_names: Sequence[str]) -> Sequence[Collection]:
    collections = []
    for name in collection_names:
        try:
            response = requests.get(f'{BASE_URL}/collections/{name}.json')
            response.raise_for_status()  
            collection_data = response.json()
            collection = Collection(**collection_data)
            collections.append(collection)
        except requests.RequestException as e:
            print(f"Error fetching collection {name}: {e}")
    return collections


relevant_collections_prompt = PromptTemplate.from_template(
    """You have access to a OGC API Features collection and should retrieve collection ID's that are relevant to a user query.
Think about what  layers needs to be included in order to perform a GIS analysis to answer the user question. 

User Query: {user_query}
Returned from /collections endpoint: {collections_endpoint}

Based on the names and Norwegian descriptions, here are the possibly relevant datasets collections (maximum of 3 collections): 
"""
)

summary_prompt = PromptTemplate.from_template(
    """Provide a summary of the relevant collections in relation to the user query: 

User Query: {user_query}
Relevant collections: {relevant_collections}

Go!
"""
)

relevant_collections_chain = relevant_collections_prompt | llm | StrOutputParser()
summary_chain = summary_prompt | llm | StrOutputParser()

complete_chain = (
    {
        'user_query': itemgetter('user_query'),
        'collections_endpoint': lambda _: requests.get('http://localhost:9000/collections').json()
    }
    | RunnablePassthrough.assign(relevant_collections=create_structured_output_runnable(RelevantCollections, llm, relevant_collections_prompt))
    | RunnablePassthrough.assign(relevant_collections=lambda x: fetch_collections(map(lambda c: c.collection_id, x['relevant_collections'])))
    | create_structured_output_runnable(CollectionDescriptions, llm, summary_prompt) 
    | RunnableLambda(lambda x: [c.dict() for c in x])
)

result = complete_chain.invoke(
    {'user_query': 'Jeg vil vite om lydforurensing nær veier'})

result

[{'collection_id': 'public.Forurensning_5001_Trondheim_25832_StoykartleggingVeg_GML',
  'description': 'Viser støysituasjonen fra vegtrafikk for de største byområdene i landet og i tillegg langs de riks- og fylkesveger der det passerer mer enn 8200 kjøretøy per døgn.',
  'geometrytype': 'Polygon',
  'relevant_properties': ['kommune',
   'støykildenavn',
   'støykilde',
   'støysonekategori',
   'støymetode',
   'beregnetÅr']},
 {'collection_id': 'public.5001elveg2.0l',
  'description': 'Elveg 2.0 - Vegnettsdatasett som omfatter alle kjørbare veger som er lengre enn 50 meter, eller del av et nettverk, samt gang- og sykkelveger og sykkelveger representert som veglenkegeometri.',
  'geometrytype': 'MultiLineString',
  'relevant_properties': ['typehinder',
   'adskiltelo',
   'fartsgrens',
   'typeveg',
   'komm',
   'adressekod',
   'adressenav',
   'vegkategor',
   'vegnummer',
   'komm_1',
   'datfgstdat']},
 {'collection_id': 'public.KartlagteFriluftslivsomr_5001_trondheim_25832_GML',


In [114]:
from pydantic import RootModel
from typing import List

class Collection(BaseModel):
    collection_id: str
    description: str
    relevant_properties: List[str]

class RelevantCollectionDescriptions(RootModel): 
    root: List[Collection]


cs = RelevantCollectionDescriptions([{'collection_id': 'public.5001elveg2.0l',
                              'description': 'Elveg 2.0 - Vegnettsdatasett som omfatter alle kjørbare veger som er lengre enn 50 meter, eller del av et nettverk, samt gang- og sykkelveger og sykkelveger representert som veglenkegeometri.',
                              'relevant_properties': ['fartsgrens']},
                             {'collection_id': 'public.Forurensning_5001_Trondheim_25832_StoykartleggingVeg_GML',
                              'description': 'Viser støysituasjonen fra vegtrafikk for de største byområdene i landet og i tillegg langs de riks- og fylkesveger der det passerer mer enn 8200 kjøretøy per døgn.',
                              'relevant_properties': []},
                             {'collection_id': 'public.Samfunnssikkerhet_5001_Trondheim_25832_Flomsoner_GML',
                              'description': 'Viser arealer som oversvømmes ved ulike fromstørrelser (gjentaksintervall).',
                              'relevant_properties': []}]
                            )

cs.model_dump_json()

'[{"collection_id":"public.5001elveg2.0l","description":"Elveg 2.0 - Vegnettsdatasett som omfatter alle kjørbare veger som er lengre enn 50 meter, eller del av et nettverk, samt gang- og sykkelveger og sykkelveger representert som veglenkegeometri.","relevant_properties":["fartsgrens"]},{"collection_id":"public.Forurensning_5001_Trondheim_25832_StoykartleggingVeg_GML","description":"Viser støysituasjonen fra vegtrafikk for de største byområdene i landet og i tillegg langs de riks- og fylkesveger der det passerer mer enn 8200 kjøretøy per døgn.","relevant_properties":[]},{"collection_id":"public.Samfunnssikkerhet_5001_Trondheim_25832_Flomsoner_GML","description":"Viser arealer som oversvømmes ved ulike fromstørrelser (gjentaksintervall).","relevant_properties":[]}]'